Relational DB model and SQL

### Connecting to Postgres server

In [1]:
import psycopg2
import json
with open('db_cred.json','r') as j:
    cred = json.load(j)
print('Connecting to the PostgreSQL database...')
conn = psycopg2.connect(
    host=cred["host"],
    port=cred['port'],
    database=cred["database"],
    user=cred["user"],
    password=cred["password"])
cur = conn.cursor()
print('PostgreSQL database version:')
cur.execute('SELECT version()')
db_version = cur.fetchone()
print(db_version)

Connecting to the PostgreSQL database...
PostgreSQL database version:
('PostgreSQL 15.2 (Debian 15.2-1.pgdg110+1) on x86_64-pc-linux-gnu, compiled by gcc (Debian 10.2.1-6) 10.2.1 20210110, 64-bit',)


### Creating Authors table

In [2]:
cc = """ CREATE TABLE authors (
                  author_id SERIAL PRIMARY KEY,
                  name VARCHAR(1000) UNIQUE NOT NULL,
                  ratings_count INTEGER NOT NULL,
                  number_of_books INTEGER NOT NULL
               )
            """

cur.execute(cc)
conn.commit()

### Inserting Data into authors table. I am inserting the author name to reduce redundancy. ratings_count to have an easier access to number of ratings the books of the author have received. number_of_books to see how many books have been published by the author.

In [3]:
import csv
from psycopg2.errors import UniqueViolation
with open('books.csv', 'r',encoding = 'utf-8') as file:
    reader = csv.reader(file)
    # Skip the header row
    next(reader)
    # Loop through each row and insert it into the database
    for row in reader:
        # Extract the values from the row
        author = row[2]
        c = row[8]
        if not author:
            continue
        else:
            insert_statement = "INSERT INTO authors (name,ratings_count,number_of_books) VALUES (%s, %s,%s);"

            try:
                cur.execute(insert_statement,(author,c,1))
                conn.commit()
            except UniqueViolation:
                conn.rollback()
                cur.execute("SELECT ratings_count FROM authors WHERE name =%s;",(author,))
                r = cur.fetchone()[0]
                cur.execute("SELECT number_of_books FROM authors WHERE name =%s;",(author,))
                n = cur.fetchone()[0]
                if c:
                    r = int(r)+ int(c)
                if n:
                    n = int(n) + 1
                cur.execute("UPDATE authors SET ratings_count = %s, number_of_books = %s WHERE name = %s",(r,n,author,))
                conn.commit()

### Creating Publishers Table

In [4]:
cc = """ CREATE TABLE publishers (
                  publisher_id SERIAL PRIMARY KEY,
                  name VARCHAR(500) UNIQUE NOT NULL,
                  number_of_pubs INTEGER NOT NULL
                )
            """

cur.execute(cc)
conn.commit()


### Inserting data into the publishers table

In [5]:
with open('books.csv', 'r',encoding = 'utf-8') as file:
    reader = csv.reader(file)
    # Skip the header row
    next(reader)
    # Loop through each row and insert it into the database
    for row in reader:
        # Extract the values from the row
        pub = row[11]
        insert_statement = "INSERT INTO publishers (name,number_of_pubs) VALUES (%s, %s);"
        try:
            cur.execute(insert_statement,(pub,1))
            conn.commit()
        except UniqueViolation:
            conn.rollback()
            cur.execute("SELECT number_of_pubs FROM publishers WHERE name =%s;",(pub,))
            n = cur.fetchone()[0]
            if n:
                n = int(n) + 1
            cur.execute("UPDATE publishers SET number_of_pubs = %s WHERE name = %s",(n,pub,))
            conn.commit()

### Creating the books table

In [6]:
createCmd = """ CREATE TABLE books (
                      book_id SERIAL PRIMARY KEY,
                      title VARCHAR(255) NOT NULL,
                      author_id INTEGER NOT NULL REFERENCES authors(author_id),
                      average_rating FLOAT,
                      isbn VARCHAR(13),
                      isbn13 VARCHAR(13),
                      language_code VARCHAR(10) NOT NULL,
                      num_pages INTEGER NOT NULL,
                      ratings_count INTEGER NOT NULL,
                      text_reviews_count INTEGER NOT NULL, 
                      publication_date DATE,
                      publisher_id INTEGER NOT NULL REFERENCES publishers(publisher_id)
                )
            """
    
cur.execute(createCmd)
conn.commit()

### Inserting data into the books table with author_id, publisher_id being foreign keys.

In [ ]:
with open('books.csv', 'r',encoding = 'utf-8') as file:
    reader = csv.reader(file)
    # Skip the header row
    next(reader)
    # Loop through each row and insert it into the database
    for row in reader:
        # Extract the values from the row
        s = 'SELECT author_id from authors where name = %s;'
        cur.execute(s,(row[2],))
        r = cur.fetchone()
        if r is not None:
            a = r[0]
        s = 'SELECT publisher_id from publishers where name = %s;'
        cur.execute(s,(row[11],))
        r = cur.fetchone()
        if r is not None:
            p = r[0]
        if row[0] != '' or row[1] != '':
            insert_statement = "INSERT INTO books (book_id,title,author_id,average_rating,isbn,isbn13,language_code,num_pages, ratings_count, text_reviews_count, publication_date, publisher_id) VALUES (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s,%s);"
            cur.execute(insert_statement,(row[0],row[1],a,row[3],row[4],row[5],row[6],row[7],row[8],row[9],row[10],p))
            conn.commit()

### Question 1 : What are the oldest 5 books in this dataset?

In [ ]:
s = 'SELECT title, publication_date FROM books ORDER BY publication_date LIMIT 5;'
cur.execute(s)
for i in cur:
    print(i)

### Question 2 : Which 5 language codes have the highest rating average?  

In [ ]:
s = 'SELECT language_code, AVG(average_rating) AS avg_rating FROM books GROUP BY language_code ORDER BY avg_rating DESC LIMIT 5;'
cur.execute(s)
for i in cur:
    print(i)

### Question 3 : Which 5 authors have the highest number of ratings? 

In [ ]:
s = 'SELECT name, ratings_count AS Number FROM authors ORDER BY ratings_count DESC LIMIT 5;'
cur.execute(s)
for i in cur:
    print(i)

### Question 4 : Which publishers publish in the highest number of different languages? 

In [ ]:
s = 'SELECT publishers.name, COUNT(DISTINCT books.language_code) AS num_languages FROM books JOIN publishers ON books.publisher_id = publishers.publisher_id GROUP BY publishers.name ORDER BY num_languages DESC LIMIT 5;'
cur.execute(s)
for i in cur:
    print(i)